In [1]:
import numpy as np 

import torch 
from torch import nn, functional as F 

In [13]:
anchor = torch.randn(2, 1, 4, requires_grad=True)
positive = torch.randn(2, 1, 4, requires_grad=True)
negative = torch.randn(2, 10, 4, requires_grad=True)

In [22]:
anchor = torch.randint(low=0, high=2, size=(1,1,4))
positive = torch.randint(low=0, high=2, size=(1,1,4))
negative = torch.randint(low=0, high=2, size=(1,10,4))

In [23]:
triplet_loss = nn.TripletMarginLoss(margin=0.1, p=2)

output = triplet_loss(anchor, positive, negative)
output

tensor(0.7197)

In [24]:
anchor

tensor([[[0, 1, 1, 0]]])

In [25]:
positive

tensor([[[1, 0, 0, 1]]])

In [26]:
negative

tensor([[[1, 0, 1, 0],
         [1, 0, 0, 1],
         [1, 1, 0, 0],
         [1, 0, 1, 0],
         [0, 1, 1, 1],
         [1, 1, 0, 1],
         [0, 0, 1, 0],
         [1, 1, 1, 1],
         [0, 1, 0, 0],
         [1, 1, 1, 1]]])

---

In [28]:
anchor_list = np.array([0, 1, 1, 0])
positive_list = np.array([1, 0, 0, 1])
negative_list = np.array([[1, 0, 1, 0],
         [1, 0, 0, 1],
         [1, 1, 0, 0],
         [1, 0, 1, 0],
         [0, 1, 1, 1],
         [1, 1, 0, 1],
         [0, 0, 1, 0],
         [1, 1, 1, 1],
         [0, 1, 0, 0],
         [1, 1, 1, 1]])

In [29]:
np.linalg.norm(anchor_list-positive_list)

2.0

In [30]:
s = 0
for i in range(10):
    s += max(np.linalg.norm(anchor_list-positive_list)-np.linalg.norm(anchor_list-negative_list[i, ...])+0.1, 0)


In [31]:
s

7.196881380565647

---

In [ ]:
negative = torch.randn(2, 10, 4, requires_grad=True)

In [90]:
feats = torch.rand(14, 128).unsqueeze(0)
batch_feat = feats.repeat(4, 1, 1)
batch_feat.shape

torch.Size([4, 14, 128])

In [55]:
ys = [[0, 1], [1, 2]]
batch_feat[ys, :] = 0

In [100]:
tuple(indexed_ys[:2, :])

(tensor([0, 4]), tensor([ 1, 13]))

In [101]:
batch_feat.shape

torch.Size([4, 14, 128])

In [112]:
edited_feat = batch_feat.index_put(indices=[torch.tensor([0, 3]), torch.tensor([6, 10])], values=torch.tensor(0.))

In [ ]:
torch.set_printoptions(threshold=100000)
print(edited_feat[1, ...])

---

In [86]:
ys = (torch.rand(32)*14).long()
ys

tensor([ 4, 13,  0, 11,  5,  6, 10,  4, 13,  0,  9, 13,  9, 11,  7,  2,  3,  9,
         5,  0,  6,  7, 10,  9,  5,  8,  4,  0,  1,  1, 13,  1])

In [87]:
feats1 = torch.rand(14, 128)

In [88]:
y_feats = feats1[ys]
y_feats.shape

torch.Size([32, 128])

In [94]:
indexed_ys = torch.stack((torch.arange(32), ys), dim=0).T
indexed_ys.shape

torch.Size([32, 2])

In [105]:
list(indexed_ys[:2, :])

[tensor([0, 4]), tensor([ 1, 13])]

---

In [116]:
def tripletLoss(pred_feat, y, feats):
    pos_feats = feats[y].unsqueeze(1)
    anchor_feat = pred_feat.unsqueeze(1)
    index_2d = [torch.arange(32), y]
    batch_feats = feats.unsqueeze(0).repeat(32, 1, 1)
    neg_feats = batch_feats.index_put(indices=index_2d, values=torch.tensor(0.))
    # print(pos_feats.shape, anchor_feat.shape, neg_feats.shape)

    triplet_loss = nn.TripletMarginLoss(margin=0.1, p=2)
    output = triplet_loss(anchor_feat, pos_feats, neg_feats)
    return output


In [117]:
pred_f = torch.rand(32, 128)
ys = (torch.rand(32)*14).long()
feats = torch.rand(14,128)

In [118]:
tripletLoss(pred_f, ys, feats)

torch.Size([32, 1, 128]) torch.Size([32, 1, 128]) torch.Size([32, 14, 128])


tensor(0.2155)

---

In [1]:
import torch 
from torch import nn, functional as F 

import numpy as np

In [2]:
ys = (torch.rand(32)*14).long()
feats = torch.rand(14,128)
pred_f = torch.rand(32, 128)
anchor_feat = feats[ys, ...]

In [3]:
anchor_feat.shape

torch.Size([32, 128])

In [14]:
bs = 32
neg_feat = torch.concat([anchor_feat[bs//2:, ...], anchor_feat[:bs//2, ...]], dim=0)
neg_feat.shape

torch.Size([32, 128])

In [13]:
neg_y = torch.concat([ys[bs//2:], ys[:bs//2]])
print("ys : ", ys, " neg ys : ", neg_y)

ys :  tensor([13,  0,  0,  2, 11,  8,  0,  4,  4,  9, 12,  4,  8, 10,  1, 13,  5, 11,
         4,  2, 11,  3,  8,  5,  7,  4,  7,  4, 12,  6, 12,  1])  neg ys :  tensor([ 5, 11,  4,  2, 11,  3,  8,  5,  7,  4,  7,  4, 12,  6, 12,  1, 13,  0,
         0,  2, 11,  8,  0,  4,  4,  9, 12,  4,  8, 10,  1, 13])


In [23]:
y_mask = (ys!=neg_y).long()
print(y_mask)

tensor([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1])


In [20]:
triplet_loss = nn.TripletMarginLoss(margin=0.1, p=2, reduction='none')

In [21]:
output = triplet_loss(pred_f, anchor_feat, neg_feat)

In [22]:
output.shape

torch.Size([32])

In [24]:
masked_output = torch.multiply(output, y_mask)
masked_output

tensor([0.8909, 0.4207, 0.0000, 0.0000, 0.0000, 0.0000, 0.1504, 0.2790, 0.0000,
        0.6632, 0.0000, 0.0000, 0.0000, 0.0000, 0.3941, 0.0052, 0.0295, 0.0000,
        0.4414, 0.0000, 0.0000, 0.7704, 0.2033, 0.4164, 0.2704, 0.0000, 0.2801,
        0.0000, 0.0000, 0.0891, 0.0000, 0.1521])

In [25]:
masked_output.mean()

tensor(0.1705)

---

In [26]:
def shuffledTripletLoss(pred_feat, anchor_feat, y, bs=32, loss_fn=nn.TripletMarginLoss(margin=0.1, p=2, reduction='none')):
    neg_feat = torch.concat([anchor_feat[bs//2:, ...], anchor_feat[:bs//2, ...]], dim=0)
    pos_feat = anchor_feat

    neg_y = torch.concat([y[bs//2:], y[:bs//2]])
    y_mask = (y!=neg_y).long()

    output_arr = loss_fn(pred_feat, pos_feat, neg_feat)
    masked_arr = torch.multiply(output_arr, y_mask)

    output = masked_arr.mean()
    return output

In [27]:
pred_f = torch.rand(32, 128)
ys = (torch.rand(32)*14).long()
feats = torch.rand(14,128)
anchor_feat = feats[ys, ...]
bs = 32 
loss_fn = nn.TripletMarginLoss(margin=0.1, p=2, reduction='none')

In [28]:
shuffledTripletLoss(pred_f, anchor_feat, ys, bs, loss_fn)

tensor(0.1424)

---

In [9]:
import torch 
from torch import nn, Tensor

In [18]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)


In [15]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [16]:
output = cos(input1, input2)
output.shape

torch.Size([100])

In [19]:
1-cos(input1, input1)

tensor([ 0.0000e+00,  5.9605e-08,  0.0000e+00,  5.9605e-08,  5.9605e-08,
         5.9605e-08,  5.9605e-08, -1.1921e-07,  0.0000e+00,  0.0000e+00,
        -1.1921e-07,  5.9605e-08,  0.0000e+00, -1.1921e-07,  0.0000e+00,
         5.9605e-08, -1.1921e-07,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         5.9605e-08,  0.0000e+00,  1.7881e-07,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.1921e-07,  1.1921e-07, -1.1921e-07,  1.1921e-07,
        -1.1921e-07, -1.1921e-07,  0.0000e+00, -1.1921e-07, -1.1921e-07,
         1.1921e-07, -1.1921e-07,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.7881e-07,  0.0000e+00,  0.0000e+00, -1.1921e-07,  0.0000e+00,
         0.0000e+00,  5.9605e-08,  0.0000e+00,  0.0000e+00, -1.1921e-07,
        -1.1921e-07,  5.9605e-08, -1.1921e-07,  0.0000e+00,  0.0000e+00,
         5.9605e-08, -1.1921e-07,  5.9605e-08,  1.1921e-07,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [7]:
l1 = nn.L1Loss(reduction="sum")

In [8]:
l1(input1, input1)

tensor(0.)